# Experiments with Wordpress Connection

Moving these out of first_experiments to clean it up.  Later might move into library to import.

In [13]:
import pandas as pd
import boto3
import boto3
import base64
from botocore.exceptions import ClientError
from datetime  import datetime, timedelta, date
import time
from urllib.parse import urlparse, parse_qs

In [2]:
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
import creds

In [4]:


# TODO: the right way to do this would be to store connection details in the secrets manager.  Unfortunately not working :(
def get_secret():

    secret_name = "data_pier"
    region_name = "ap-southeast-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )


    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # I've deleted a chunk of error handling bumpf

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )


def get_data_pier_connection():
    #db_pass = input()
    db_pass = creds.pg_pass
    db_host = "data-pier-production.cl8qfdl47mtr.ap-southeast-1.rds.amazonaws.com"
    db_user = "jamie"
    db_database = "data_pier"
    pg_conn = psycopg2.connect(host=db_host, user = db_user, dbname=db_database, password= db_pass)
    pg_conn.set_session(readonly=True, autocommit=True) #without this it can lock tables.
    
    
    return pg_conn, pg_conn.cursor()

In [5]:
pg_conn, cursor = get_data_pier_connection()
cursor.execute("select * from information_schema.tables limit 10")

In [6]:
#query = "select * from information_schema.columns where table_schema='id_wordpress'" 
query = "select  * from id_wordpress.wp_terms join id_wordpress.wp_posts where slug like '%kartu%' limit 100" # gets slug
query = """
        select * from id_wordpress.wp_postmeta where meta_key not like'%oembed%' 
        and meta_key like '%slug%'
        order by meta_key desc limit 100
        """

query = "select taxonomy, count(*) from id_wordpress.wp_term_taxonomy group by taxonomy"
query = "select * from id_wordpress.wp_term_taxonomy where taxonomy='post_tag'"
pd.read_sql_query(query, pg_conn).head()

,term_taxonomy_id,term_id,taxonomy,description,parent,count
0,577.0,577.0,post_tag,,0.0,137
1,582.0,582.0,post_tag,,0.0,20
2,583.0,583.0,post_tag,,0.0,49
3,586.0,586.0,post_tag,,0.0,123
4,591.0,591.0,post_tag,,0.0,58


In [24]:
def get_all_categories_tags(conn, schema):
    
    """domain.com/<slug> (we can parse the slug based on URL single post format)
Get the post from slug
```sql
SELECT wp_posts.*
FROM wp_posts 
WHERE 1=1 
AND wp_posts.post_name = '<slug>'
AND wp_posts.post_type = 'post' 
ORDER BY wp_posts.post_date DESC
```
Get categories of a post (post_id is from post retrieved above). For tag, replace 'category' by 'tag'
```sql
SELECT t.term_id, tt.parent, tt.count, tt.taxonomy
FROM wp_terms AS t 
INNER JOIN wp_term_taxonomy AS tt
ON t.term_id = tt.term_id
INNER JOIN wp_term_relationships AS tr
ON tr.term_taxonomy_id = tt.term_taxonomy_id
WHERE tt.taxonomy IN ('category')
AND tr.object_id IN (<post_id>)
ORDER BY t.name ASC
```
Get details of a term (category or tag) from IDs retrieved above
```sql
SELECT t.*, tt.*
FROM wp_terms AS t
INNER JOIN wp_term_taxonomy AS tt
ON t.term_id = tt.term_id
WHERE t.term_id = <term_id>
```"""
    
    
    tags_categories_query = """
    select 
        wp_posts.id as post_id, 
        -- wp_posts.post_name as slug, 
        cats.name, 
        cats.term_id, 
        cats.parent, 
        cats.count, 
        cats.taxonomy
    
    from {schema}.wp_posts
    join (
        SELECT tr.object_id, t.term_id, tt.parent, tt.count, tt.taxonomy, t.name
        FROM {schema}.wp_terms AS t 
        INNER JOIN {schema}.wp_term_taxonomy AS tt
        ON t.term_id = tt.term_id
        INNER JOIN {schema}.wp_term_relationships AS tr
        ON tr.term_taxonomy_id = tt.term_taxonomy_id
         WHERE tt.taxonomy IN ('category', 'post_tag')
       
        
    ) as cats
    on cats.object_id = wp_posts.id
    
    where
    wp_posts.post_type = 'post'
    and wp_posts.post_status = 'publish'
    
    """.format(schema=schema)

    
    
    tags_categories = pd.read_sql_query(tags_categories_query, conn)
    #print(tags_categories[:100])
    
    print(tags_categories.head())
    tags_categories.astype({"post_id":int, "term_id":int, "parent":int, "taxonomy": "category"}) #untested type casting
    
    
    tags = tags_categories[tags_categories.taxonomy=='post_tag'][['post_id','name']]
    
    categories = tags_categories[tags_categories.taxonomy=='category'][['post_id','name']]
    
    
    return categories, tags


def get_post_data(conn, schema):
    query = """
    select id as post_id, post_name as slug, post_author as author_id, CHAR_LENGTH(post_content) as post_num_chars, post_date, post_modified 
    from {schema}.wp_posts
    
    where 
    wp_posts.post_type = 'post'
    and wp_posts.post_status = 'publish'
    """.format(schema=schema)
    df = pd.read_sql_query(query, conn)
    df.astype({"post_id":int, "author_id":int}) #untested type casting
    return df
    
def slug_to_url(slug):
    return "https://www.moneysmart.id/"+slug



post_data = get_post_data(pg_conn, "id_wordpress")
    
post_slugs_to_id_df = post_data[["slug","post_id",]].set_index(["slug",])

categories, tags = get_all_categories_tags(pg_conn, "id_wordpress")

   post_id                       name  term_id  parent  count  taxonomy
0    153.0                   Pinjaman    528.0  1180.0    510  category
1    155.0                  Budgeting   1193.0  1191.0    633  category
2    155.0                 tips hemat   1299.0     0.0    179  post_tag
3    155.0  mengatur keuangan pribadi   1944.0     0.0      5  post_tag
4    155.0              mengatur uang   3053.0     0.0      5  post_tag


In [35]:
post_data.head()

,post_id,slug,author_id,post_num_chars,post_date,post_modified
0,153.0,cara-cepat-melunasi-kpr,58.0,11264,2013-05-10 14:17:47,2016-06-08 20:13:01
1,155.0,mengatur-keuangan-pribadi,2617.0,4365,2013-05-14 18:47:59,2019-03-22 11:53:08
2,158.0,kpr-rumah-dan-cara-memilihnya,2617.0,1959,2013-05-16 11:06:36,2019-03-22 11:29:39
3,161.0,agar-otak-selalu-segar,0.0,1652,2013-05-23 12:46:14,2013-05-23 12:46:14
4,199.0,mewujudkan-impian-membeli-mobil-murah,0.0,3079,2013-10-21 10:30:47,2013-10-21 10:30:47


In [42]:
print("approx size of total post data in mega-chars: %.2f " % (sum(post_data["post_num_chars"])/1.0e6 ) )
print("Gives an idea of whether going to run out of memory")

approx size of total post data in mega-chars: 94.79 
Gives an idea of whether going to run out of memory


In [25]:
post_slugs_to_id_df.head()


,post_id
slug,
cara-cepat-melunasi-kpr,153.0
mengatur-keuangan-pribadi,155.0
kpr-rumah-dan-cara-memilihnya,158.0
agar-otak-selalu-segar,161.0
mewujudkan-impian-membeli-mobil-murah,199.0


In [26]:
def convert_canonical_url_to_slug(url):
    if "//" not in url:
        raise Exception("expected start of url in {}".format(url))
    id_root = "https://www.moneysmart.id"
    #ms_id_index = url.find(id_root)
    if not url.startswith(id_root):
        raise Exception("{url} doesn't look like a canonical url".format(url = url))
        
    #slug = url[ms_id_index+ len(id_root)+1:].strip("/")
    parsed_url = urlparse(url)
    slug = parsed_url.path.strip("/")
    
    if slug == "":
        slug ="/"
    return slug
        
def is_article_slug(slug):
    if "/" in slug or len(slug)==0:
        return False
    return True

def get_post_id_from_slug(slug):
    post_id = post_slugs_to_id_df.loc[slug, :][0]
    return post_id


def get_post_id_from_url(url):
    #Expects only post urls
    slug = convert_canonical_url_to_slug(url)
    print(slug)
    post_id = get_post_id_from_slug(slug)
    return post_id

In [29]:
get_post_id_from_url("https://www.moneysmart.id/cara-cepat-melunasi-kpr/")

cara-cepat-melunasi-kpr


153.0

In [31]:
categories.head()

,post_id,name
0,153.0,Pinjaman
1,155.0,Budgeting
6,158.0,Pinjaman
10,161.0,Ragam
11,171.0,Kuliner &amp; Travel


In [32]:
# article_categories = categories.groupby('slug').aggregate(lambda x:x.unique().tolist())
# article_tags = tags.groupby('slug').aggregate(lambda x:x.unique().tolist())

In [33]:
#[z for z in article_categories.values if len(z[1])>0]
# article_categories
# article_categories['num_categories'] = article_categories['name'].apply(len)

In [34]:
#TODO: sometime should do all this based on post_id

def get_categories_tags_for_url(slug):
    #cats = categories[categories.slug==slug]
    
    post_id = get_post_id_from_slug(slug)
    tags = 
    
    """
    try:
        tags = article_tags.loc[slug, :][1]
    except:
        print("no tags")
        tags = []
    try:
        cats = article_categories.loc[slug, :][1]
    except:
        print("no categories for %s"%slug)
        cats = []
        """
    return cats, tags

def get_category_tags_for_url(slug):
    """
    this returns just the first category (and maybe not even the primary category... yet)
    It's useful for grouping easily by category without duplication.
    """
    cats, tags = get_categories_tags_for_slug()

start_time = time.time()
cats, tags = get_categories_tags_for_url("pembalap-indonesia-berkarier-di-mancanegara")
end_time = time.time()

print("it took ")
print(end_time-start_time)
cats_tags

no tags
no categories for pembalap-indonesia-berkarier-di-mancanegara
it took 
0.001224517822265625


NameError: name 'cats_tags' is not defined

# Post Data

Do some pretty simple parsing using beautifulsoup:

* Word count
* Number of characters
* Number of links
* Number of tables
* Number of uls
* Numboer of lists
* Number of h2s
* Youtube embeds?

And add some regular metadata (should be in data frames):

* First published
* Last update
* Author

In [ ]:
class PostBodyStats(object):
    def __init__(self, post_body):
        self.word_count = self.extract_word_count(post_body)
        self.character_count = self.extract_character_count(post_body)
        self.links_count_total = self.extract_links_count(post_body)
    
    def extract_word_count(self, post_body):
        
        >>
        
    def extract_number_of_characters(self, post_body)

## Dump to CSV for Sharing